# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def run_insert(insert_query, data):
    try:
        cur.execute(insert_query, data)
        conn.commit()
    except psycopg2.Error as e: 
        print("Error: Query: ", insert_query, "\nRows for data: ", data)
        print (e)
        raise e

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("./data/song_data")

In [5]:
def load_songs_to_db_from_dataframe(song_data_df):
    song_data = song_data_df[["song_id", "title", "artist_id", "year", "duration"]]
    for i, row in song_data.iterrows():
        run_insert(song_table_insert, row)
        
def load_artists_to_db_from_dataframe(song_data_df):
    artist_data = song_data_df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
    for i, row in artist_data.iterrows():
        run_insert(artist_table_insert, row)

def load_artists_and_songs_from_file(file):
    song_data_df = pd.read_json(file, lines=True)
    load_songs_to_db_from_dataframe(song_data_df)
    load_artists_to_db_from_dataframe(song_data_df)

In [6]:
for file in song_files:
    load_artists_and_songs_from_file(file)

In [7]:
filepath = song_files[0]

In [8]:
song_data_df = pd.read_json(filepath, lines=True)
song_data_df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [54]:
# song_id, title, artist_id, duration
song_data = song_data_df[["song_id", "title", "artist_id", "year", "duration"]]
song_data
#song_data = song_data.values[0]
#song_data = list(song_data)
#song_data

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [37]:
for i, row in song_data_df.iterrows():
    cur.execute(song_table_insert, row)
    conn.commit()

TypeError: not all arguments converted during string formatting

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [29]:
# artist_id, artist_name, artist_location, artist_latitude, artist_longitude
artist_data = song_data_df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data = artist_data.values[0]
artist_data = list(artist_data)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [9]:
log_files = get_files("./data/log_data")

In [10]:
def timestamp_to_row(timestamp):
    dt = pd.to_datetime(timestamp,unit='ms')
    return (dt, dt.hour, dt.day, dt.week, dt.month, dt.year, dt.weekday())

def load_time_to_db_from_dataframe(log_data_df):
    df_time = log_data_df[log_data_df["page"] == "NextSong"] 
    column_labels = ('start_time','hour','day','week','month','year','weekday')
    time_data = list(df_time['ts'].apply(timestamp_to_row))
    time_df = pd.DataFrame(time_data, columns=column_labels)
    for i, row in time_df.iterrows():
        run_insert(time_table_insert, list(row))
        
def load_user_to_db_from_dataframe(log_data_df):
    user_df = log_data_df[["userId", "firstName", "lastName", "gender", "level"]]
    for i, row in user_df.iterrows():
        run_insert(user_table_insert, row)
        
def load_songplay_from_logdata_dataframe(log_data_df):
    for index, row in log_data_df.iterrows():
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()

        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
        # insert songplay record
        # start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
        ts = pd.to_datetime(row.ts, unit='ms')
        songplay_data = (ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
        
def log_file_to_sanitized_dataframe(file):
    log_data_df = pd.read_json(file, lines=True)
    log_data_df['userId'].replace('', np.nan, inplace=True)
    log_data_df = log_data_df[log_data_df.userId.notnull()]
    return log_data_df

def process_log_files(log_files):
    for file in log_files:
        log_data_df = log_file_to_sanitized_dataframe(file)
        load_time_to_db_from_dataframe(log_data_df)
        load_user_to_db_from_dataframe(log_data_df)
        load_songplay_from_logdata_dataframe(log_data_df)

In [11]:
process_log_files(log_files)

In [7]:
filepath = log_files[0]

In [8]:
log_data_df = pd.read_json(filepath, lines=True)
log_data_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [9]:
df_time = log_data_df[log_data_df["page"] == "NextSong"] 
df_time.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [10]:
t = pd.to_datetime(df_time['ts'],unit='ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [11]:
#  timestamp, hour, day, week of year, month, year, and weekday 
# or per schema: start_time, hour, day, week, month, year, weekday
column_labels = ('start_time','hour','day','week','month','year','weekday')
def timestamp_to_row(timestamp):
    dt = pd.to_datetime(timestamp,unit='ms')
    return (dt, dt.hour, dt.day, dt.week, dt.month, dt.year, dt.weekday())
time_data = list(df_time['ts'].apply(timestamp_to_row))

In [9]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

NameError: name 'time_data' is not defined

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [13]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [14]:
user_df = log_data_df[["userId", "firstName", "lastName", "gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [15]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [16]:
for index, row in log_data_df.iterrows():
    print(row.song, ",", row.artist,",", row.length)

None , None , nan
None , None , nan
You Gotta Be , Des'ree , 246.30812
None , None , nan
Flat 55 , Mr Oizo , 144.03873
Quem Quiser Encontrar O Amor , Tamba Trio , 177.18812
Eriatarka , The Mars Volta , 380.42077
Becoming Insane , Infected Mushroom , 440.2673
Congratulations , Blue October / Imogen Heap , 241.3971
Once again , Girl Talk , 160.15628
Pump It , Black Eyed Peas , 214.93506
None , None , nan
Nobody Puts Baby In The Corner , Fall Out Boy , 200.72444
Mango Pickle Down River (With The Wilcannia Mob) , M.I.A. , 233.7171
Eye Of The Tiger , Survivor , 245.36771


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
def insert_songplay_from_logdata_dataframe(log_data_df):
    for index, row in log_data_df.iterrows():

        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()

        if results:
            songid, artistid = results
            print(results, " : ", row.artist, " : ", row.song, row.length)
        else:
            songid, artistid = None, None
        # insert songplay record
        songplay_data = ()
        #cur.execute(songplay_table_insert, songplay_data)
        #conn.commit()

None  :  Stephen Lynch  :  Jim Henson's Dead
None  :  Manowar  :  Shell Shock
None  :  Morcheeba  :  Women Lose Weight (Feat: Slick Rick)
None  :  Maroon 5  :  Won't Go Home Without You
None  :  Train  :  Hey_ Soul Sister
None  :  LMFAO  :  I'm In Miami Bitch
None  :  DJ Dizzy  :  Sexy Bitch
None  :  Fish Go Deep & Tracey K  :  The Cure & The Cause (Dennis Ferrer Remix)
None  :  None  :  None
None  :  M83  :  Staring At Me
None  :  Saybia  :  The Second You Sleep
None  :  Local Natives  :  Wide Eyes
None  :  South Park  :  La Resistance (Medley) (LP Version)
None  :  UNKLE Feat. Josh Homme  :  Restless
None  :  Justyna Steczkowska  :  Noc...
None  :  Evanescence  :  Like You
None  :  Coldplay  :  God Put A Smile Upon Your Face
None  :  Hot Chip  :  Playboy
None  :  Ellie Goulding  :  Every Time You Go
None  :  Postal Service  :  Natural Anthem (Album)
None  :  None  :  None
None  :  None  :  None
None  :  Jack Johnson  :  Taylor
None  :  Iron And Wine  :  Naked As We Can
None  :  The x

None  :  Ashanti  :  VooDoo
None  :  Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner  :  Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)
None  :  Clint Eastwood  :  Frankie's Office
None  :  Charttraxx Karaoke  :  Fireflies
None  :  Jason Mraz  :  Summer Breeze
None  :  The B-52's  :  Love Shack
None  :  Aventura  :  Alexandra
None  :  Five Iron Frenzy  :  Canada
None  :  12 Stones  :  Anthem For The Underdog
None  :  Kings Of Leon  :  Revelry
None  :  Brooklyn Bounce  :  Get Ready To Bounce Recall 08
None  :  Dashboard Confessional  :  The Good Fight
None  :  None  :  None
None  :  Lonnie Johnson  :  Low Land Moan
None  :  None  :  None
None  :  Leggo Beast  :  In
None  :  The Pussycat Dolls  :  Sway
None  :  Stereophonics  :  I Miss You Now
None  :  The Beautiful South  :  Masculine Eclipse
None  :  The Acacia Strain  :  As If Set Afire
None  :  ATB  :  Underwater World
None  :  The Robert Cray Band  :  Our Last Time
None  :  Enigma  :  T.N.

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [213]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.